# Aggregate Interaction and association data into a Network
* Known interactions are setup within a global network:
    * Drug-Drug interactions (DDI)
        * DrugBank
        * Continuous Drug Combination Database (CDCDB)
    * Drug-Protein interactions (DPI)
        * DrugBank
    * Drug-SNP interactions
        * DrugBank
    * Protein-SNP associations
        * UniProt
        * dbSNP
    * Protein-Phenotype interactions (a.k.a genotype-phenotype)
        * MIM

For best results, run the following notebooks to ensure extracted data is up to date:
* For UniProt data: [ExtractAnnotationsData_UniProt](../database/ExtractAnnotationsData_UniProt.ipynb)
* For DrugBank data: [ExtractAnnotationsData_DrugBank](../database/ExtractAnnotationsData_DrugBank.ipynb)
* For SNP data: [ExtractData_dbSNP](../database/ExtractData_dbSNP.ipynb)
* For MIM data: [ExtractAnnotationsData_MIM](../database/ExtractAnnotationsData_MIM.ipynb)

Once the basic interaction structure is created, various node and attributes can be extracted from the relevant sources 
It is expected that this notebook will grow over time and extract additional relevant information for nodes/edges to create a knowledge-graph setup.
## Setup
### Import packages

In [ ]:
from collections import defaultdict

import pandas as pd
from rbc_gem_utils import (
    GEM_NAME,
    build_string,
    get_annotation_df,
    get_dirpath,
    read_cobra_model,
    show_versions,
    split_string,
)
from rbc_gem_utils.database import CDCDB_DB_TAG, DRUGBANK_DB_TAG, MIM_DB_TAG
from rbc_gem_utils.util import explode_column, strip_plural

show_versions()

## Load RBC-GEM model

In [ ]:
model_dirpath = get_dirpath("model")
model = read_cobra_model(filename=model_dirpath / f"{GEM_NAME}.xml")
model

In [ ]:
annotation_type = "genes"
df_model_mappings = get_annotation_df(
    getattr(model, annotation_type),
    [
        "uniprot",
        "hgnc.symbol",
        "drugbank",
        "dbsnp",
        "mim",
        "ec-code",
        "orphanet",
        "pubchem.compound",
    ],
).rename({"id": annotation_type}, axis=1)
for col in df_model_mappings.columns:
    df = df_model_mappings[col].apply(split_string).explode(col).drop_duplicates()
    print(f"{df.name}: {df.nunique()}")

df_model_mappings

### Setup node alias table and data dictionaries

In [ ]:
# Edge data
interaction_dfs = defaultdict(list)

protein_node_id_type = "uniprot"
df_network_node_aliases = df_model_mappings[
    [protein_node_id_type, protein_node_id_type]
].copy()
df_network_node_aliases.columns = ["node_id", "node_alias"]
df_network_node_aliases["node_source"] = protein_node_id_type
df_network_node_aliases["node_type"] = "gene/protein"
df_network_node_aliases["alias_type"] = "identifier"
for key, alias_type in zip(["genes", "hgnc.symbol"], ["identifier", "symbol"]):
    df = df_model_mappings.loc[:, [protein_node_id_type, key]]
    if key != "genes":
        df["node_source"] = key
    else:
        df["node_source"] = GEM_NAME
        df[key] = df[key].apply(lambda x: f"G_{x}")
    df["node_type"] = "gene/protein"
    df["alias_type"] = alias_type

    df_network_node_aliases = pd.concat(
        (
            df_network_node_aliases,
            df.rename(
                {
                    protein_node_id_type: "node_id",
                    key: "node_alias",
                },
                axis=1,
            ),
        )
    )
node_types_keys = {
    "drug": "drugbank",
    "snp": "dbsnp",
}
for node_type, key in node_types_keys.items():
    list_of_ids = sorted(
        df_model_mappings[key].apply(split_string).explode(key).dropna().unique()
    )
    df = pd.DataFrame.from_dict(
        {
            "node_id": list_of_ids,
            "node_alias": list_of_ids,
            "node_source": [key] * len(list_of_ids),
            "node_type": [node_type] * len(list_of_ids),
            "alias_type": "identifier",
        },
        orient="columns",
    )
    df_network_node_aliases = pd.concat(
        (
            df_network_node_aliases,
            df,
        )
    )

# Note that node aliases at this point may include nodes that never get connected. Nodes are pruned after edges are created
node_types = (
    df_network_node_aliases.groupby("node_type")["node_alias"]
    .agg(lambda x: set(x.unique()))
    .to_dict()
)
df_network_node_aliases

## Load interaction data
#### Load drug interactions
##### With other drugs 

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database", DRUGBANK_DB_TAG) / f"Interactions_{DRUGBANK_DB_TAG}.tsv",
    sep="\t",
    index_col=0,
    dtype=str,
)
# Add names to aliases
df = pd.concat(
    (
        df_interactions.loc[:, ["drugbank_A", "name_A"]].rename(
            {"drugbank_A": "node_id", "name_A": "node_alias"}, axis=1
        ),
        df_interactions.loc[:, ["drugbank_B", "name_B"]].rename(
            {"drugbank_B": "node_id", "name_B": "node_alias"}, axis=1
        ),
    ),
    axis=0,
).drop_duplicates()
df["node_source"] = "drugbank"  # Is INN?
df["node_type"] = "drug"
df["alias_type"] = "name"

df_network_node_aliases = (
    pd.concat(
        (
            df_network_node_aliases,
            df,
        ),
        axis=0,
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
df_network_node_aliases

# Add Drug Drug mappings
edge_type = "drug-drug"
rename_mapping = {
    "drugbank_A": "node_A",
    "drugbank_B": "node_B",
}

df_interactions = df_interactions.loc[:, list(rename_mapping)].rename(
    rename_mapping, axis=1
)
df_interactions = df_interactions[df_interactions["node_A"].isin(node_types["drug"])]
df_interactions = df_interactions[df_interactions["node_B"].isin(node_types["drug"])]
df_interactions = df_interactions.drop_duplicates().reset_index(drop=True)
df_interactions["edge_type"] = edge_type
df_interactions["edge_subtype"] = float("nan")
df_interactions["edge_source"] = "drugbank"
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

##### With proteins

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database", DRUGBANK_DB_TAG) / f"Proteins_{DRUGBANK_DB_TAG}.tsv",
    sep="\t",
    index_col=0,
    dtype=str,
)
edge_type = "drug-gene/protein"
# Add Drug Protein mappings
rename_mapping = {
    "drugbank-id": "node_A",
    "polypeptide.uniprot-id": "node_B",
    "type": "edge_subtype",
}
df_interactions = df_interactions.loc[:, list(rename_mapping)].rename(
    rename_mapping, axis=1
)
df_interactions = df_interactions[df_interactions["node_A"].isin(node_types["drug"])]
df_interactions = df_interactions[
    df_interactions["node_B"].isin(node_types["gene/protein"])
]
df_interactions = df_interactions.drop_duplicates().reset_index(drop=True)
df_interactions["edge_type"] = edge_type
df_interactions["edge_source"] = "drugbank"
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

##### With SNPs

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database", DRUGBANK_DB_TAG) / f"SNP_{DRUGBANK_DB_TAG}.tsv",
    sep="\t",
    index_col=0,
    dtype=str,
)
edge_type = "drug-snp"
df_interactions["edge_subtype"] = df_interactions["adverse-reaction"].notna()

# Add Drug SNP mappings
rename_mapping = {
    "drugbank-id": "node_A",
    "rs-id": "node_B",
    "edge_subtype": "edge_subtype",
}
df_interactions = df_interactions.loc[:, list(rename_mapping)].rename(
    rename_mapping, axis=1
)
df_interactions = df_interactions[df_interactions["node_A"].isin(node_types["drug"])]
df_interactions = df_interactions[df_interactions["node_B"].isin(node_types["snp"])]
df_interactions = df_interactions.drop_duplicates().reset_index(drop=True)
df_interactions["edge_type"] = edge_type
df_interactions["edge_source"] = "drugbank"
df_interactions["edge_subtype"] = df_interactions["edge_subtype"].apply(
    lambda x: "known-effect-adverse" if x else "known-effect"
)
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

#### Load protein SNPs

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database") / f"snps_{GEM_NAME}.tsv", sep="\t", index_col=0, dtype=str
)
df_model_snps = (
    df_model_mappings.set_index("uniprot")["dbsnp"]
    .apply(split_string)
    .explode()
    .dropna()
    .reset_index(drop=False)
)

edge_type = "gene/protein-snp"
df_interactions = df_model_snps.merge(
    df_interactions, left_on="dbsnp", right_on="dbsnp", how="inner"
)
rename_mapping = {"uniprot": "node_A", "dbsnp": "node_B", "snp_class": "edge_subtype"}
df_interactions = df_interactions.loc[:, list(rename_mapping)].rename(
    rename_mapping, axis=1
)
df_interactions = df_interactions[
    df_interactions["node_A"].isin(node_types["gene/protein"])
]
df_interactions = df_interactions[df_interactions["node_B"].isin(node_types["snp"])]
df_interactions = df_interactions.drop_duplicates().reset_index(drop=True)
df_interactions["edge_type"] = edge_type
df_interactions["edge_source"] = build_string(["uniprot", "dbsnp"])
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

### Load MIM genotypes/phenotypes

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database", MIM_DB_TAG) / f"{MIM_DB_TAG}_{GEM_NAME}.tsv",
    sep="\t",
    index_col=None,
    dtype=str,
)
df_interactions = pd.merge(
    df_model_mappings.set_index("genes")[[protein_node_id_type]],
    df_interactions,
    left_index=True,
    right_on="genes",
).reset_index(drop=True)
df_interactions = df_interactions.loc[
    :,
    [
        "genes",
        "uniprot",
        "mim",
        "title",
        "mim.phenotype",
        "mim.phenotype.title",
        "mim.phenotype.title.symbol",
    ],
]
# Add MIM aliases for gene/protein
rename_mapping = {protein_node_id_type: "node_id", "mim": "node_alias"}
df = (
    df_interactions.loc[:, list(rename_mapping)]
    .rename(rename_mapping, axis=1)
    .dropna()
    .drop_duplicates()
)
df["node_alias"] = df["node_alias"].apply(lambda x: f"mim:{x}")
df["node_source"] = "mim"
df["node_type"] = "gene/protein"
df["alias_type"] = "identifier"
df_network_node_aliases = (
    pd.concat(
        (
            df_network_node_aliases,
            df,
        ),
        axis=0,
    )
    .drop_duplicates()
    .reset_index(drop=True)
)

# Add MIM symbols as aliases for phenotypes
rename_mapping = {
    "mim.phenotype": "node_id",
    "mim.phenotype.title.symbol": "node_alias",
}
df = (
    df_interactions.loc[:, list(rename_mapping)]
    .rename(rename_mapping, axis=1)
    .dropna(subset=["node_id"])
    .drop_duplicates()
)
df["node_source"] = "mim"
df["node_type"] = "phenotype/disease"
df["alias_type"] = "symbol"
df_network_node_aliases = (
    pd.concat(
        (
            df_network_node_aliases,
            df.copy(),
        ),
        axis=0,
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
# Also record the node ID alias
df["node_alias"] = df["node_id"]
df["alias_type"] = "identifier"
df_network_node_aliases = (
    pd.concat(
        (
            df_network_node_aliases,
            df.copy(),
        ),
        axis=0,
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
# Add edge type
edge_type = "gene/protein-phenotype/disease"
rename_mapping = {
    "uniprot": "node_A",
    "mim.phenotype": "node_B",
}
df_interactions = df_interactions.loc[:, list(rename_mapping)].rename(
    rename_mapping, axis=1
)
df_interactions = df_interactions.dropna().drop_duplicates().reset_index(drop=True)
df_interactions["edge_type"] = edge_type
df_interactions["edge_source"] = "mim"
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

### Load effective drug combinations
* If both PubChem and DrugBank are used, harmonization of data will be necessary
* Combinations with more than two drug components are split into combination pairs

In [ ]:
df_interactions = pd.read_csv(
    get_dirpath("database", CDCDB_DB_TAG) / f"EffectiveCombos_{CDCDB_DB_TAG}.tsv",
    sep="	",
    index_col=None,
    dtype=str,
)
edge_type = "drug-drug"
df_interactions = df_interactions[
    df_interactions["node_A"].apply(
        lambda x: x.startswith("DB") if isinstance(x, str) else False
    )
]
df_interactions = df_interactions[
    df_interactions["node_B"].apply(
        lambda x: x.startswith("DB") if isinstance(x, str) else False
    )
]
df_interactions = df_interactions.loc[
    :, ["node_A", "node_B", "edge_type", "edge_subtype", "edge_source"]
]
df_interactions = df_interactions.loc[
    df_interactions.loc[:, ["node_A", "node_B"]]
    .apply(lambda x: build_string(sorted(x.values)), axis=1)
    .drop_duplicates()
    .index,
    :,
]
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

### Load model as nodes/edges
Reactions, metabolites, genes all included, pseudoreactions excluded.

In [ ]:
to_exclude = {"Pseudoreactions"}

# Genes already have aliases previously
gene_to_uniprot = df_model_mappings.set_index("genes")[protein_node_id_type].to_dict()

edge_type = "gene/protein-reaction"
df_interactions = pd.DataFrame.from_dict(
    {
        gene_to_uniprot[gene.id]: build_string(
            [
                reaction.id
                for reaction in list(gene.reactions)
                if reaction.subsystem not in to_exclude
            ]
        )
        for gene in model.genes
    },
    orient="index",
    columns=["node_B"],
)
df_interactions.index.name = "node_A"
df_interactions = explode_column(df_interactions, name="node_B").reset_index(drop=False)
df_interactions["edge_type"] = edge_type
df_interactions["edge_source"] = GEM_NAME
# Prefix node for clarity
df_interactions["node_B"] = df_interactions["node_B"].apply(lambda x: f"R_{x}")
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

In [ ]:
# Reactions and metabolites need aliases, can be added here.
edge_type = "reaction-metabolite"
df_interactions = pd.DataFrame.from_dict(
    {
        reaction.id: build_string([m.id for m in reaction.metabolites])
        for reaction in model.reactions
        if reaction.subsystem not in to_exclude
    },
    orient="index",
    columns=["node_B"],
)
df_interactions.index.name = "node_A"
df_interactions = explode_column(df_interactions, name="node_B").reset_index(drop=False)
df_interactions["edge_type"] = edge_type
df_interactions["edge_subtype"] = df_interactions.apply(
    lambda x: (
        "reactant"
        if model.metabolites.get_by_id(x["node_B"])
        in model.reactions.get_by_id(x["node_A"]).reactants
        else "product"
    ),
    axis=1,
)
df_interactions["edge_source"] = GEM_NAME

# Prefix node for clarity
df_interactions["node_A"] = df_interactions["node_A"].apply(lambda x: f"R_{x}")
df_interactions["node_B"] = df_interactions["node_B"].apply(lambda x: f"M_{x}")
interaction_dfs[edge_type] += [df_interactions.copy()]
df_interactions

In [ ]:
attribute_types_and_aliases = {
    "metabolites": ["kegg.compound", "metatlas"],
    "reactions": ["kegg.reaction", "metatlas", "ec-code"],
}

for attribute_type, annotations_for_aliases in attribute_types_and_aliases.items():
    if attribute_type == "reactions":
        prefix = "R_"
        obj_list = getattr(model, attribute_type).query(
            lambda x: x.subsystem not in to_exclude
        )
    else:
        prefix = "M_"
        obj_list = getattr(model, attribute_type)
    df = get_annotation_df(obj_list, annotations_for_aliases)
    df["id"] = df["id"].apply(lambda x: f"{prefix}{x}")
    df["node_id"] = df["id"]
    df = df.rename({"id": f"model.{attribute_type}"}, axis=1)
    # Add model metabolite names to aliases
    attribute = "name"
    df[f"model.{attribute_type}.{attribute}"] = df[f"model.{attribute_type}"].apply(
        lambda x: getattr(obj_list.get_by_id(x[len(prefix) :]), attribute)
    )
    df = df.melt(id_vars="node_id", var_name="node_source", value_name="node_alias")
    df["alias_type"] = "identifier"

    # Clean up attribute alias types
    index = df[df["node_source"].str.startswith(f"model.{attribute_type}.")].index
    df.loc[list(index), "alias_type"] = df.loc[list(index), "node_source"].apply(
        lambda x: x.split(".", 2)[-1]
    )
    df.loc[list(index), "node_source"] = GEM_NAME
    # Set node type
    df["node_type"] = strip_plural(attribute_type)

    df_network_node_aliases = (
        pd.concat(
            (
                df_network_node_aliases,
                df,
            ),
            axis=0,
        )
        .drop_duplicates()
        .reset_index(drop=True)
    )
df_network_node_aliases

### Aggregate data for nodes and edges

In [ ]:
df_network_edges = pd.DataFrame(columns=["node_A", "node_B", "edge_type"])
interaction_dfs_concat = {
    key: pd.concat(df_list, axis=0) if len(df_list) > 1 else df_list[0]
    for key, df_list in interaction_dfs.items()
}
interaction_dfs_final = {}

### Harmonize data
#### Drug-Drug interactions

In [ ]:
edge_type = "drug-drug"
df_interactions = interaction_dfs_concat[edge_type]
df_interactions["nodes"] = df_interactions[["node_A", "node_B"]].apply(
    lambda x: build_string(sorted(x)), axis=1
)

df_harmonized = df_interactions[df_interactions["nodes"].duplicated(keep=False)]
df_harmonized = df_harmonized.groupby("nodes", as_index=False)[
    ["edge_type", "edge_subtype", "edge_source"]
].agg(lambda x: ";".join(sorted(x.dropna().unique())))
df_harmonized["node_A"] = df_harmonized["nodes"].apply(lambda x: x.split(";")[0])
df_harmonized["node_B"] = df_harmonized["nodes"].apply(lambda x: x.split(";")[-1])

df_interactions = pd.concat(
    (df_interactions[~df_interactions["nodes"].duplicated(keep=False)], df_harmonized),
    axis=0,
)
df_interactions = df_interactions.loc[
    :, ["node_A", "node_B", "edge_type", "edge_subtype", "edge_source"]
]
interaction_dfs_final[edge_type] = df_interactions
df_interactions

In [ ]:
for edge_type, df in interaction_dfs_concat.items():
    if edge_type not in interaction_dfs_final:
        interaction_dfs_final[edge_type] = df

In [ ]:
df_network_edges = pd.DataFrame(columns=["node_A", "node_B", "edge_type"])
for df in interaction_dfs_final.values():
    df_network_edges = pd.concat((df_network_edges, df), axis=0)
df_network_node_aliases = df_network_node_aliases.dropna(
    subset=["node_alias"]
).reset_index(drop=True)
df_network_nodes = (
    df_network_node_aliases.loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

all_edge_node_ids = set(df_network_edges["node_A"]).union(df_network_edges["node_B"])
df_isolated = (
    df_network_node_aliases[~df_network_node_aliases["node_id"].isin(all_edge_node_ids)]
    .loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
)
df_network_nodes = (
    df_network_node_aliases[df_network_node_aliases["node_id"].isin(all_edge_node_ids)]
    .loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
)
df_edge_summary = df_network_edges.set_index(["node_A", "node_B"])[
    "edge_type"
].value_counts()
df_node_summary = df_network_nodes.set_index(["node_id"])["node_type"].value_counts()
df_isolated_summary = df_isolated.set_index(["node_id"])["node_type"].value_counts()

print(df_edge_summary, f"\n\nTotal number of edges:\t {df_edge_summary.sum()}\n")
print(df_node_summary, f"\n\nTotal number of nodes:\t {df_node_summary.sum()}\n")
print(
    df_isolated_summary,
    f"\n\nTotal number of isolated nodes:\t {df_isolated_summary.sum()}\n",
)

## Export data

In [ ]:
df_network_edges.to_csv(
    get_dirpath("database") / f"NetworkEdges_{GEM_NAME}.tsv",
    sep="\t",
)
df_network_node_aliases.to_csv(
    get_dirpath("database") / f"NetworkNodes_{GEM_NAME}.tsv",
    sep="\t",
)

## Visualize global interaction network
TODO find a better color scheme and additional representations

In [ ]:
from operator import itemgetter

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

In [ ]:
df_network_edges = pd.read_csv(
    get_dirpath("database") / f"NetworkEdges_{GEM_NAME}.tsv",
    sep="\t",
    index_col=0,
    dtype=str,
)
df_network_node_aliases = pd.read_csv(
    get_dirpath("database") / f"NetworkNodes_{GEM_NAME}.tsv",
    sep="\t",
    index_col=0,
    dtype=str,
)

df_network_node_aliases = df_network_node_aliases.dropna(
    subset=["node_alias"]
).reset_index(drop=True)
df_network_nodes = (
    df_network_node_aliases.loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

all_edge_node_ids = set(df_network_edges["node_A"]).union(df_network_edges["node_B"])
df_isolated = (
    df_network_node_aliases[~df_network_node_aliases["node_id"].isin(all_edge_node_ids)]
    .loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
)
df_network_nodes = (
    df_network_node_aliases[df_network_node_aliases["node_id"].isin(all_edge_node_ids)]
    .loc[:, ["node_id", "node_type"]]
    .drop_duplicates()
)
df_edge_summary = df_network_edges.set_index(["node_A", "node_B"])[
    "edge_type"
].value_counts()
df_node_summary = df_network_nodes.set_index(["node_id"])["node_type"].value_counts()
df_isolated_summary = df_isolated.set_index(["node_id"])["node_type"].value_counts()

print(df_edge_summary, f"\n\nTotal number of edges:\t {df_edge_summary.sum()}\n")
print(df_node_summary, f"\n\nTotal number of nodes:\t {df_node_summary.sum()}\n")
print(
    df_isolated_summary,
    f"\n\nTotal number of isolated nodes:\t {df_isolated_summary.sum()}\n",
)

In [ ]:
cmap = mpl.colormaps["jet"].resampled(len(df_node_summary.index))
node_colors_dict = {
    node_type: mpl.colors.to_hex(cmap(idx))
    for idx, node_type in enumerate(df_node_summary.index)
}

cmap = mpl.colormaps["viridis"].resampled(len(df_edge_summary.index))
edge_colors_dict = {
    node_type: mpl.colors.to_hex(cmap(idx))
    for idx, node_type in enumerate(df_edge_summary.index)
}
# node_colors_dict = {
#     "drug": mpl.colors.to_hex("xkcd:red"),
#     "snp":  mpl.colors.to_hex("xkcd:blue"),
#     "gene/protein":  mpl.colors.to_hex("xkcd:yellow"),
#     "phenotype/disease":  mpl.colors.to_hex("xkcd:pink"),
#     "reaction": mpl.colors.to_hex("xkcd:green"),
#     "metabolite": mpl.colors.to_hex("xkcd:purple"),
# }
# edge_colors_dict = {
#      "drug-drug": mpl.colors.to_hex("xkcd:red"),
#      "drug-gene/protein": mpl.colors.to_hex("xkcd:purple"),
#      "drug-snp": mpl.colors.to_hex("xkcd:orange"),
#      "gene/protein-snp": mpl.colors.to_hex("xkcd:green"),
#      "gene/protein-phenotype/disease": mpl.colors.to_hex("xkcd:magenta"),
#      "gene/protein-reaction": mpl.colors.to_hex("xkcd:light blue"),
#      "reaction-metabolite": mpl.colors.to_hex("xkcd:light blue"),
# }
include_isolated = True
fig, ax_nodes = plt.subplots(1, 1, figsize=(10, 5))
sns.despine(fig)
barsize = 0.5
edgecolor = "black"
edgewidth = 0.8


df_nodes = (
    pd.concat((df_node_summary, df_isolated_summary), axis=1).fillna(0).astype(int)
)
df_nodes.columns = ["Connected", "Isolated"]
table_cell_text = []

# include_isolated = False
offset = np.zeros(len(df_nodes.index))
for idx, (label, series) in enumerate(df_nodes.items()):
    if idx == 1:
        # Lighten the color for isolated nodes
        colors = [
            mpl.colors.to_hex(mpl.colors.to_rgba_array(node_colors_dict[k]) / 2 + 0.5)
            for k in series.index
        ]
    else:
        colors = [node_colors_dict[k] for k in series.index]
    rects = ax_nodes.bar(
        series.index,
        series.values,
        barsize,
        bottom=offset,
        color=colors,
        edgecolor=edgecolor,
        linewidth=edgewidth,
    )
    offset += series.values
    if not include_isolated and idx == 0:
        break
ax_nodes.set_xticks([])
ax_nodes.set_xticklabels([])
ax_nodes.set_ylabel("Number of nodes")


for node_type_target in df_nodes.index:
    row_text = []
    for node_type_source in df_nodes.index:
        key = f"{node_type_source}-{node_type_target}"
        try:
            value = df_edge_summary[key]
        except KeyError:
            try:
                value = df_edge_summary[f"{node_type_target}-{node_type_source}"]
            except KeyError:
                value = ""
        row_text.append(str(value))
    table_cell_text.append(row_text)
table = ax_nodes.table(
    table_cell_text,
    rowLabels=df_nodes.index,
    colLabels=df_nodes.index,
    loc="bottom",
    cellLoc="center",
    rowLoc="center",
    colLoc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(9)
ax_nodes.annotate(
    "# of Interactions",
    xy=(-0.1, -0.03),
    xycoords=ax_nodes.transAxes,
    fontsize=9,
    ha="center",
)
fig.tight_layout()
fig;

In [ ]:
# df_nodes = df_network_nodes.copy()
# df_edges = df_network_edges.copy()

# df_nodes["color"] = df_nodes["node_type"].apply(lambda x: node_colors_dict[x])
# df_edges["color"] = df_edges["edge_type"].apply(lambda x: edge_colors_dict[x])

# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# nx_graph = nx.from_pandas_edgelist(
#     df=df_edges,
#     source="node_A",
#     target="node_B",
#     create_using=nx.Graph,
#     edge_attr=True,
# )
# nx_graph.add_nodes_from(
#     (n, dict(d))
#     for n, d in df_nodes.set_index("node_id").iterrows()
#     if n in nx_graph.nodes
# )

# node_colors = [attr_dict.get("color", "grey") for (node_id, attr_dict) in nx_graph.nodes(data=True)]
# edge_colors = [attr_dict.get("color", "grey") for (x, y, attr_dict) in nx_graph.edges(data=True)]

# pos = nx.circular_layout(nx_graph, scale=1, center=None, dim=2)
# nx.draw_networkx_nodes(
#     nx_graph,
#     pos,
#     ax=ax,
#     # nodelist=None,
#     # node_size=300,
#     node_color=node_colors,
#     # node_shape='o',
#     # alpha=None,
#     # cmap=None,
#     # vmin=None,
#     # vmax=None,
#     # linewidths=None,
#     # edgecolors=None,
#     # label=None,
#     # margins=None,
# )
# nx.draw_networkx_edges(
#     nx_graph,
#     pos,
#     ax=ax,
#     # edgelist=None,
#     # width=1.0,
#     edge_color=edge_colors,
#     # style='solid',
#     # alpha=None,
#     # arrowstyle=None,
#     # arrowsize=10,
#     # edge_cmap=None,
#     # edge_vmin=None,
#     # edge_vmax=None,
#     # arrows=None,
#     # label=None,
#     # node_size=300,
#     # nodelist=None,
#     # node_shape='o',
#     # connectionstyle='arc3',
#     # min_source_margin=0,
#     # min_target_margin=0,
# )
# fig;